In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.auto_mturk import (create_mturk_client,clean_own_worker,
                                 Turker,MTurkparam,get_batch_indexes)
from src.data.watcher import Watcher

from src.visualization.turk_widgets import get_control_panel
from src.data.auto_drive import get_drive_service, download_drive_txt, download_all_csv_results,download_drive_spreadsheet
from src.data.fraudulous import detect_repeat_frauders,detect_honey_frauders
from src.utils import generate_password 
from src.constants import (WATCHER_PATH,NMB_FORMS_THRESHOLD,TOKEN_PATH,
                           CREDS_PATH, URL_INDEX_PATH, AWS_KEYS_PATH,
                           FORMS_RESULTS_DIR,WATCHER_FORMS_RESULTS_DIR,HONEYPOTS)
from src.utils import read_access_keys
import subprocess
import pandas as pd
import jdc
from IPython.display import clear_output
import pickle as pk
from src.constants import HONEYPOTS
import Levenshtein
import pickle as pk

import ipywidgets as widgets
from src.utils import get_form_urls

## Urls info retrieval

<div class="alert alert-success">
Adapt the following variables if you want a specific batch size/ batch number. If BATCH_NUMBER is set to <b>None</b>, the batch number is automatically computed. 
</div>

In [30]:
BATCH_SIZE = 3
BATCH_NUMBER = 0

In [25]:
# retrieve the indexes for the next batch to compute
batch_number, form_indexes = get_batch_indexes(batch_number=BATCH_NUMBER,
                                               batch_size=BATCH_SIZE)

In [28]:
file_id = "1NkiTTW_Dvahg6l-e_bMNC2elA3lFbcXvPz2lFfOKW78"
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
formidx2url,formidx2gid = get_form_urls(service,file_id)
formidx2url = {key:value for key,value in formidx2url.items() if key in form_indexes}
formidx2gid = {key:value for key,value in formidx2gid.items() if key in form_indexes}

Download 100%


In [29]:
formidx2gid

{0: '1OPwW9IcXnpNEuOF5txTSck5cXtSl8NEhAAXV1OyKRSs',
 1: '1oIvkJ05QQFpxTQ7-YEU-HXLhOjzkI4vS5qWjBwPr2p0',
 2: '1kn960ZoLCI0uN1qtUAMk06wJGD9WRNkQ0ZI_L03AxGA'}

In [32]:
download_all_csv_results(formidx2gid,FORMS_RESULTS_DIR,service)

## MTurk management

In [1]:
param = MTurkparam(MaxAssignments = 4,
             LifetimeInSeconds = 172800,       #
             AutoApprovalDelayInSeconds=86400, # time after submission before autoapproval
             AssignmentDurationInSeconds=600,  # max time to perform the assignment
             Reward='0.3',
             n_forms=len(formidx2gid),
             max_forms_per_worker=2,
             production=True)

NameError: name 'MTurkparam' is not defined

In [15]:
turk = Turker(param=param,
              gservice=service,
              formidx2url=formidx2url,
              formidx2gid=formidx2gid,
              formrespath=FORMS_RESULTS_DIR)

Loading hit2form


In [17]:
get_control_panel(turk,file_id)

Text(value='', placeholder='Results HITid/formidx')

## Cleaning workers 

In [30]:
monitor = Watcher(".",formidx2url,service,production=True)
monitor.clean_all_workers("3N5C8MI2ZCLZ0AAT5UVXEVWWHP8G22")

In [24]:
a = turk.client.list_reviewable_hits()['HITs']

In [30]:
turk.hit2form

{'3WUVMVA7OBOVA6EUHPVJTAA5SA2ZAZ': 0, '3HYV4299H0HJGNUJO4XSC93SEGU8EZ': 1}

In [71]:
exworkers = set()
qualifs = turk.client.list_workers_with_qualification_type(QualificationTypeId="3OR1BBO28PIVPWZMRDTWE8U6OZXNGN")
for qualif in qualifs['Qualifications']:
    if qualif['QualificationTypeId'] == "3OR1BBO28PIVPWZMRDTWE8U6OZXNGN":
        exworkers.add(qualif['WorkerId'])

RequestError: An error occurred (RequestError) when calling the ListWorkersWithQualificationType operation: QualificationType 3OR1BBO28PIVPWZMRDTWE8U6OZXNGN does not exist. (1606845848334)

https://workersandbox.mturk.com/mturk/

In [22]:
clean_own_worker(turk.client,"3OR1BBO28PIVPWZMRDTWE8U6OZXNGN")

Worker already clean


## Qualifications

In [12]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=0
)
# production 3N5C8MI2ZCLZ0AAT5UVXEVWWHP8G22
# sandbox 3OR1BBO28PIVPWZMRDTWE8U6OZXNGN

{'QualificationType': {'QualificationTypeId': '3N5C8MI2ZCLZ0AAT5UVXEVWWHP8G22',
  'CreationTime': datetime.datetime(2020, 12, 1, 10, 35, 46, tzinfo=tzlocal()),
  'Name': 'emojidone',
  'Description': 'Qualification to recognize workers that already performed an emoji-related task',
  'QualificationTypeStatus': 'Active',
  'IsRequestable': True,
  'AutoGranted': True,
  'AutoGrantedValue': 0},
 'ResponseMetadata': {'RequestId': 'e7843690-3baf-4459-8b16-a9ee51eaf8ec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e7843690-3baf-4459-8b16-a9ee51eaf8ec',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '318',
   'date': 'Tue, 01 Dec 2020 09:35:45 GMT'},
  'RetryAttempts': 0}}

In [11]:
turk.client.get_qualification_type(QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN')

RequestError: An error occurred (RequestError) when calling the GetQualificationType operation: QualificationType 3OR1BBO28PIVPWZMRDTWE8U6OZXNGN does not exist. (1606815318089)

In [ ]:
turker.client.update_qualification_type(
    QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN',
    QualificationTypeStatus='Active',
    Test='string',
    AnswerKey='string',
    TestDurationInSeconds=123,
    RetryDelayInSeconds=123,
    AutoGranted=False,
    AutoGrantedValue=123
)